In [ ]:
import nltk
import jieba
import pandas as pd
import numpy as np
from pymongo import MongoClient
import re
from collections import Counter
import math

In [ ]:
def get_data(host, database):
    
    """ 連接DB 
    """    
    client = MongoClient(host)
    db = client[database]
    collection = db.article
    print(collection.stats)
    """ 拿到data
    """
    cursor = collection.find()
    data = pd.DataFrame(list(cursor))
    
    """ 製作contents, corpus
        用jieba.lcut返回list
        濾掉符號
    """
    content = data['contents']
    corpus = []
    for i in range(len(content)):
        corpus.append(jieba.lcut(re.sub('[#！、，「」（）。《》()【】]','',content[i])))
    
    """ 製作corpus list
    """
    corpus_list = []
    for j in range(len(corpus)):
        tmp = {'id':data._id.values[j],
               'corpus':corpus[j]}
        corpus_list.append(tmp)

    print('finish')
    
    return data, corpus, content, corpus_list

In [ ]:
host = host
database = database

data, corpus, content, corpus_list = get_data(host, database)

In [ ]:
def make_tfidf(corpora):
     
    corpus = np.array(corpora['corpus'])
    _id = np.array(corpora['id'])
       
    count = Counter(corpus)
   
    """ TF
    """
    tf = np.array(list(count.values())) / len(corpus)

    """ IDF
    """    
    word = np.array(list(count.keys()))
    count = np.array([np.zeros(len(word))])
    
    for i in range(len(content)):
        contents = content[i]
        tmp = np.array(list(map(lambda x: 1 if x in contents else 0,word)))
        count = count + tmp
        
    count_in_corpus = count[0]    
    idf = list(map(lambda x: math.log(len(content)/(x+1)), count_in_corpus))
    
    """ TF-IDF
    """
    tf_idf = tf * idf

    """ dict
    """
    tmp_dict = pd.DataFrame( { '_id': _id,
                 'key': word, 
                 'tf-idf':tf_idf}  )    

    return tmp_dict

In [ ]:
""" 全部的tfidf
    裝入一個list
"""
tfidf_list = list(map(lambda x: make_tfidf(x),corpus_list))

In [ ]:
""" 篩出前15的詞
"""
top15 = list(map(lambda x: x.sort_values(['tf-idf'],ascending=False).head(15).reset_index(), tfidf_list))